In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport
from lightgbm import LGBMRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder
import datetime
from sklearn.metrics import mean_squared_error, mean_absolute_error
import gc
import os

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-feb-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-feb-2021/test.csv')
sub = pd.read_csv('../input/tabular-playground-series-feb-2021/sample_submission.csv')

In [ ]:
columns = test.columns[1:]
columns

In [ ]:
target = train['target'].values

In [ ]:
cat_features = columns[:10]
cat_features

In [ ]:
for feature in cat_features:
    le = LabelEncoder()
    le.fit(train[feature])
    train[feature] = le.transform(train[feature])
    test[feature] = le.transform(test[feature])

In [ ]:
train.head()

In [ ]:
test.columns

In [ ]:
test.columns

In [ ]:
cont_features = test.columns[11:25]
cont_features

In [ ]:
from sklearn.preprocessing import LabelEncoder
alldata = pd.concat([train[cont_features],test[cont_features]],axis=0)
alldata.shape
n=2

max_cat_values = []
for c in cont_features:
    le = LabelEncoder()
    x = le.fit_transform(alldata[c].round(n))
    train[c+'_cat'] = le.transform(train[c].round(n))
    test[c+'_cat'] = le.transform(test[c].round(n))
    max_cat_values.append(np.max(x)+1)
    
n=1

max_cat_values3 = []
for c in cont_features:
    le = LabelEncoder()
    x = le.fit_transform(alldata[c].round(n))
    train[c+'_cat3'] = le.transform(train[c].round(n))
    test[c+'_cat3'] = le.transform(test[c].round(n))
    max_cat_values3.append(np.max(x)+1)
    
n=10

max_cat_values4 = []
for c in cont_features:
    alldata[c] = pd.cut(alldata[c],bins=n,labels=np.arange(n))
    train[c+'_cat4'] = alldata.iloc[:len(train)][c].astype("int32")
    test[c+'_cat4'] = alldata.iloc[len(train):][c].astype("int32")
    max_cat_values4.append(np.max(alldata[c])+1)

In [ ]:
test.head()

In [ ]:
train_oof = np.zeros((300000,))
test_preds = 0
train_oof.shape

In [ ]:
params = {'reg_alpha': 6.147694913504962,
 'reg_lambda': 0.002457826062076097,
 'colsample_bytree': 0.3,
 'subsample': 0.8,
 'learning_rate': 0.0005,
 'max_depth': 20,
 'num_leaves': 111,
 'min_child_samples': 285,
 'random_state': 48,
 'n_estimators': 320000,
 'metric': 'rmse',
 'cat_smooth': 39}

In [ ]:
from tensorflow.keras.layers import Input,Dense,Dropout
from tensorflow.keras import Model
from  tensorflow.keras.regularizers import l2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,BatchNormalization,Dropout,Embedding,Flatten,Concatenate
from keras import backend as K
import tensorflow as tf
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
def get_model():
    inputs = []
    flatten_layers = []
    for e, c in enumerate(cont_features):
        input_c = Input(shape=(1, ), dtype='int32')
        num_c = max_cat_values[e]
        embed_c = Embedding(
            num_c,
            3,
        )(input_c)
        embed_c = Dropout(0.6)(embed_c)
        flatten_c = Flatten()(embed_c)
        inputs.append(input_c)
        flatten_layers.append(flatten_c)
        
    flatten_layers2 = []
    for e, c in enumerate(cont_features):
        input_c = Input(shape=(1, ), dtype='int32')
        num_c = max_cat_values3[e]
        embed_c = Embedding(
            num_c,
            12,
        )(input_c)
        embed_c = Dropout(0.5)(embed_c)
        flatten_c = Flatten()(embed_c)
        inputs.append(input_c)
        flatten_layers2.append(flatten_c)
        
    flatten_layers3 = []
    for e, c in enumerate(cont_features):
        input_c = Input(shape=(1, ), dtype='int32')
        num_c = max_cat_values4[e]
        embed_c = Embedding(
            num_c,
            12,
        )(input_c)
        embed_c = Dropout(0.5)(embed_c)
        flatten_c = Flatten()(embed_c)
        inputs.append(input_c)
        flatten_layers3.append(flatten_c)

    input_num = Input(shape=(14,), dtype='float32')
    inputs.append(input_num)
    
    # 1st embeddings with main continous features.
    flatten_layers.append(input_num) # This line is important. I'm concatenating original features with the first type of categorical features' embeddings.
    flatten = Concatenate()(flatten_layers)
    x = Dense(1024, activation='relu',bias_initializer="normal",kernel_regularizer=regularizers.l2(0.0001),
              bias_regularizer=regularizers.l2(0.00000),
             activity_regularizer = regularizers.l2(0.0001))(flatten) 
    x = Dropout(0.)(x)
    x = Dense(1024, activation='relu',kernel_initializer='normal')(x) 
    
    # 2nd embeddings.
    flatten2 = Concatenate()(flatten_layers2)
    x2 = Dense(2048, activation='relu',bias_initializer="normal")(flatten2) 
    x2 = Dense(2048, activation='relu',kernel_initializer='normal')(x2) # 1500 original
    
    # 3rd embeddings.
    flatten3 = Concatenate()(flatten_layers3)
    x3 = Dense(2048, activation='relu',bias_initializer="normal")(flatten3) 
    x3 = Dense(2048, activation='relu',kernel_initializer='normal')(x3)
    
    
    x = Concatenate()([x,x2,x3]) # Concating all there outputs.
  
    
    outputs = Dense(1, activation='linear')(x)
    model = Model(inputs=inputs, outputs=outputs)
    opt = tf.keras.optimizers.Adam(learning_rate=0.001)
    model.compile(loss="mse", optimizer=opt)
    return model

In [ ]:
model = get_model()
model.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else: 
        return lr * 0.95

In [ ]:
features = [col for col in train.columns if col not in ["target","id","preds","scaled_target","lgbm_preds"]][-14:]
len(features)

In [ ]:
features

In [ ]:
#X = X.abs()
y = train["target"]
kf = KFold(n_splits=5, shuffle=True, random_state=1)
oof = np.zeros(len(train))
score_list = []
fold = 1
test_preds = []

    
for train_index, test_index in kf.split(train):
    X_train, X_val = train.iloc[train_index], train.iloc[test_index]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]
    

    # Preparing model inputs.
    X_train_list=[]
    X_val_list = []
    test_list = []
    for col in cont_features:
        X_train_list.append(X_train[col+'_cat'].values)
        X_val_list.append(X_val[col+'_cat'].values)
        test_list.append(test[col+'_cat'].values)
    for col in cont_features:
        X_train_list.append(X_train[col+'_cat3'].values)
        X_val_list.append(X_val[col+'_cat3'].values)
        test_list.append(test[col+'_cat3'].values)
    for col in cont_features:
        X_train_list.append(X_train[col+'_cat4'].values)
        X_val_list.append(X_val[col+'_cat4'].values)
        test_list.append(test[col+'_cat4'].values)
        
    X_train_list.append(X_train[cont_features].values)
    X_val_list.append(X_val[cont_features].values)
    test_list.append(test[cont_features].values)

    y_pred_list = []

    # Callbacks
    estop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
    lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
    
    model = get_model()
    history = model.fit(X_train_list, y_train,validation_data=(X_val_list,y_val), 
              callbacks=[lr_callback,estop], epochs=10, batch_size=1024, verbose=1)
   
   
        
    y_pred_list.append(model.predict(X_val_list).ravel())
    oof[test_index] = np.mean(y_pred_list,axis=0)
    score = np.sqrt(mean_squared_error(y_val, oof[test_index]))
    score_list.append(score)
    print(f"RMSE Fold-{fold} : {score}")
    
    test_preds.append(model.predict(test_list).ravel())
    
    fold+=1

np.mean(score_list)

In [ ]:
print(np.mean(score_list))
score_list

In [ ]:
train["4_preds"] = oof
test["4_preds"] = np.mean(test_preds,axis=0)
np.sqrt(mean_squared_error(train["target"], train["4_preds"]))

In [ ]:
train

In [ ]:
train[["id","4_preds"]].to_csv("nn_preds_train.csv",index=False)
test[["id","4_preds"]].to_csv("nn_preds_test.csv",index=False)

In [ ]:
# train = train.rename(columns={'4_preds': 'target'})
# train
test = test.rename(columns={'4_preds': 'target'})
# train

In [ ]:
train[["id","target"]].to_csv("nn_preds_train.csv",index=False)
test[["id","target"]].to_csv("nn_preds_test.csv",index=False)

In [ ]:
NUM_FOLDS = 1
kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=0)

for f, (train_ind, val_ind) in tqdm(enumerate(kf.split(train, target))):
        #print(f'Fold {f}')
        train_df, val_df = train.iloc[train_ind][columns], train.iloc[val_ind][columns]
        train_target, val_target = target[train_ind], target[val_ind]
        
        model = LGBMRegressor(**params)
        model.fit(train_df, train_target, eval_set=[(val_df,val_target)],early_stopping_rounds=1600,verbose=False)
        temp_oof = model.predict(val_df)
        temp_test = model.predict(test[columns])

        train_oof[val_ind] = temp_oof
        test_preds += temp_test/NUM_FOLDS
        
        print(mean_squared_error(temp_oof, val_target, squared=False))

In [ ]:
mean_squared_error(train_oof, target, squared=False)

In [ ]:
np.save('train_oof', train_oof)
np.save('test_preds', test_preds)

In [ ]:
sub['target'] = test_preds
sub.to_csv('submission.csv', index=False)